# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 10:34AM,246141,20,8515444,"Exact-Rx, Inc.",Released
1,Sep 12 2022 10:34AM,246141,20,8515445,"Exact-Rx, Inc.",Released
2,Sep 12 2022 10:34AM,246141,20,8515446,"Exact-Rx, Inc.",Released
3,Sep 12 2022 10:25AM,246140,19,ADV80005556,"AdvaGen Pharma, Ltd",Executing
4,Sep 12 2022 10:25AM,246140,19,ADV80005557,"AdvaGen Pharma, Ltd",Executing
5,Sep 12 2022 10:25AM,246140,19,ADV80005558,"AdvaGen Pharma, Ltd",Released
6,Sep 12 2022 10:25AM,246140,19,ADV80005559,"AdvaGen Pharma, Ltd",Released
7,Sep 12 2022 10:25AM,246140,19,ADV80005560,"AdvaGen Pharma, Ltd",Released
8,Sep 12 2022 10:25AM,246131,20,ACI-8217699-BO,"ACI Healthcare USA, Inc.",Released
9,Sep 12 2022 10:25AM,246131,20,8506334,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,246137,Released,9
24,246138,Released,4
25,246140,Executing,2
26,246140,Released,3
27,246141,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246135,NaN,6.0
246137,NaN,9.0
246138,NaN,4.0
246140,2.0,3.0
246141,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246060,0.0,1.0
246096,0.0,1.0
246100,1.0,0.0
246103,6.0,1.0
246106,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246060,0,1
246096,0,1
246100,1,0
246103,6,1
246106,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246060,0,1
1,246096,0,1
2,246100,1,0
3,246103,6,1
4,246106,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246060,,1
1,246096,,1
2,246100,1,
3,246103,6,1
4,246106,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 10:34AM,246141,20,"Exact-Rx, Inc."
3,Sep 12 2022 10:25AM,246140,19,"AdvaGen Pharma, Ltd"
8,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc."
43,Sep 12 2022 10:20AM,246138,10,Bio-PRF
47,Sep 12 2022 10:19AM,246137,10,Bio-PRF
56,Sep 12 2022 10:18AM,246135,10,Eywa Pharma Inc.
62,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc."
63,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc."
64,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC"
66,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 12 2022 10:34AM,246141,20,"Exact-Rx, Inc.",,3
1,Sep 12 2022 10:25AM,246140,19,"AdvaGen Pharma, Ltd",2,3
2,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",,35
3,Sep 12 2022 10:20AM,246138,10,Bio-PRF,,4
4,Sep 12 2022 10:19AM,246137,10,Bio-PRF,,9
5,Sep 12 2022 10:18AM,246135,10,Eywa Pharma Inc.,,6
6,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",,1
7,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",,1
8,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",,2
9,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:34AM,246141,20,"Exact-Rx, Inc.",3,
1,Sep 12 2022 10:25AM,246140,19,"AdvaGen Pharma, Ltd",3,2
2,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",35,
3,Sep 12 2022 10:20AM,246138,10,Bio-PRF,4,
4,Sep 12 2022 10:19AM,246137,10,Bio-PRF,9,
5,Sep 12 2022 10:18AM,246135,10,Eywa Pharma Inc.,6,
6,Sep 12 2022 10:09AM,246134,19,"GUSA Granules USA, Inc.",1,
7,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",1,
8,Sep 12 2022 9:57AM,246127,15,"Carwin Pharmaceutical Associates, LLC",2,
9,Sep 12 2022 9:55AM,246100,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:34AM,246141,20,"Exact-Rx, Inc.",3,
1,Sep 12 2022 10:25AM,246140,19,"AdvaGen Pharma, Ltd",3,2
2,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",35,
3,Sep 12 2022 10:20AM,246138,10,Bio-PRF,4,
4,Sep 12 2022 10:19AM,246137,10,Bio-PRF,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:34AM,246141,20,"Exact-Rx, Inc.",3.0,NaN
1,Sep 12 2022 10:25AM,246140,19,"AdvaGen Pharma, Ltd",3.0,2.0
2,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",35.0,NaN
3,Sep 12 2022 10:20AM,246138,10,Bio-PRF,4.0,NaN
4,Sep 12 2022 10:19AM,246137,10,Bio-PRF,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 10:34AM,246141,20,"Exact-Rx, Inc.",3.0,0.0
1,Sep 12 2022 10:25AM,246140,19,"AdvaGen Pharma, Ltd",3.0,2.0
2,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",35.0,0.0
3,Sep 12 2022 10:20AM,246138,10,Bio-PRF,4.0,0.0
4,Sep 12 2022 10:19AM,246137,10,Bio-PRF,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2707365,175.0,14.0
12,246113,2.0,0.0
15,738377,34.0,0.0
19,738377,5.0,8.0
20,738378,39.0,0.0
21,738256,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2707365,175.0,14.0
1,12,246113,2.0,0.0
2,15,738377,34.0,0.0
3,19,738377,5.0,8.0
4,20,738378,39.0,0.0
5,21,738256,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,175.0,14.0
1,12,2.0,0.0
2,15,34.0,0.0
3,19,5.0,8.0
4,20,39.0,0.0
5,21,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,175.0
1,12,Released,2.0
2,15,Released,34.0
3,19,Released,5.0
4,20,Released,39.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,14.0,0.0,0.0,8.0,0.0,1.0
Released,175.0,2.0,34.0,5.0,39.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,14.0,0.0,0.0,8.0,0.0,1.0
1,Released,175.0,2.0,34.0,5.0,39.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,14.0,0.0,0.0,8.0,0.0,1.0
1,Released,175.0,2.0,34.0,5.0,39.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()